In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

Vamos supor que você queira fazer um modelo que rankeia os tickets de suporte dos consumidores por prioridade e direciona-os para o departamento correto.O seu modelo possui três entradas:


* o titulo do ticket (text input)
* o texto do ticket (text input)
* qualquer rótulo adicionado pelo consumidor (categorical input, assumido aqui como um one-hot encoded)

Nós podemos codificar as entradas de texto como arrays de um e zeros de tamanhos de vocabulário

seu modelo também tem duas saídas:

* o score de prioridade do ticket, um escalar entre 0 e 1 (sigmoid output)
* o departamento que tem que receber (softmax sobre o conjunto de departamento)

construindo usando functional API

In [2]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

title = keras.Input(shape=(vocabulary_size,),name='title') # define a entrada do modelo
text_body = keras.Input(shape=(vocabulary_size,),name='text_body') #define a entrada do modelo
tags = keras.Input(shape=(num_tags,),name='tags') # define a entrada do modelo

features = keras.layers.Concatenate()([title,text_body,tags]) # combina as features de entrada em um unico tensor concatenando-os
features = keras.layers.Dense(64,activation='relu')(features) # aplica uma camada intermediaria para recombina as entradas de features para uma representação mais rica

priority = keras.layers.Dense(1,activation='sigmoid',name='priority')(features) # saida do modelo

department = keras.layers.Dense(num_departments,activation='softmax',name='department')(features) # saida do modelo

model = keras.Model(inputs=[title,text_body,tags],outputs=[priority,department]) # cria o modelo

2024-12-24 16:22:15.693314: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-12-24 16:22:15.693381: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: mickael-Aspire-A515-54G
2024-12-24 16:22:15.693388: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: mickael-Aspire-A515-54G
2024-12-24 16:22:15.693492: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 550.120.0
2024-12-24 16:22:15.693511: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 550.120.0
2024-12-24 16:22:15.693516: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 550.120.0


In [4]:
# dummy input variables
num_samples = 1280
title_data = np.random.randint(0,2,size=(num_samples,vocabulary_size))
text_body_data = np.random.randint(0,2,size=(num_samples,vocabulary_size))
tags_data = np.random.randint(0,2,size=(num_samples,num_tags))

#dummy outputs variables

priority_data = np.random.random(size=(num_samples,1))
department = np.random.random(size=(num_samples,num_departments))

model.compile(loss=['mse','categorical_crossentropy'], optimizer='adam',metrics=[['mse'],['accuracy']])

model.fit([title_data,text_body_data,tags_data],[priority_data,department],epochs=1)

model.evaluate([title_data,text_body_data,tags_data],[priority_data,department])

priority_preds,department_preds = model.predict([title_data,text_body_data,tags_data])

2024-12-24 16:31:55.258268: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102400000 exceeds 10% of free system memory.
2024-12-24 16:31:55.382453: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102400000 exceeds 10% of free system memory.


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - department_accuracy: 0.2475 - loss: 29.9454 - priority_mse: 0.2938


2024-12-24 16:31:57.780987: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102400000 exceeds 10% of free system memory.
2024-12-24 16:31:57.971017: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102400000 exceeds 10% of free system memory.


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - department_accuracy: 0.2845 - loss: 19.6248 - priority_mse: 0.3343


2024-12-24 16:31:58.547985: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102400000 exceeds 10% of free system memory.


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


uma outra maneira de compilar usando dicionarios que é bem melhor

In [5]:
model.compile(optimizer='adam',loss={'priority':'mse','department':'categorical_crossentropy'},metrics={'priority':'mse','department':'accuracy'})
model.fit({'title':title_data,'text_body':text_body_data,'tags':tags_data},{'priority':priority_data,'department':department})
priority_preds, department_preds = model.predict({'title': title_data,'text_body': text_body_data,'tags':tags_data})

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - department_accuracy: 0.2762 - loss: 30.3612 - priority_mse: 0.2924
40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 17ms/step


In [6]:
keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) for `plot_model` to work.
